In [2]:
import os
import re
from tqdm import tqdm
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Dense, Dropout, LSTM, Embedding, Bidirectional
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.callbacks import TensorBoard
from sklearn.model_selection import train_test_split
from keras.layers import Embedding
from  keras . utils  import  to_categorical
from keras.layers import Dense, Input, GlobalMaxPooling1D
from keras.layers import Conv1D,Conv2D, MaxPooling1D, Embedding, Flatten
from keras.models import Model
from keras.initializers import Constant
from keras.layers import Embedding


import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [4]:
import pandas as pd 
train_data=pd.read_csv('gdrive/My Drive/Thesis Data/Our dataset/OurDataset_train.csv')
stopwords=pd.read_csv('gdrive/My Drive/Colab Notebooks/Stopwords.csv')
test_data=pd.read_csv('gdrive/My Drive/Thesis Data/Our dataset/OurDataset_test.csv')


In [5]:
train_data.head

<bound method NDFrame.head of                      label  ... is_valid
0      shunil_gongopaddhay  ...    False
1            humayun_ahmed  ...    False
2                 shomresh  ...    False
3            humayun_ahmed  ...    False
4            humayun_ahmed  ...    False
...                    ...  ...      ...
14369         robindronath  ...    False
14370        humayun_ahmed  ...    False
14371                  MZI  ...    False
14372             shomresh  ...    False
14373           shirshendu  ...    False

[14374 rows x 3 columns]>

In [6]:
!git clone -l -s https://github.com/banglakit/bengali-stemmer.git

Cloning into 'bengali-stemmer'...
remote: Enumerating objects: 25, done.
remote: Counting objects: 100% (25/25), done.
remote: Compressing objects: 100% (17/17), done.
remote: Total 94 (delta 5), reused 16 (delta 4), pack-reused 69
Unpacking objects: 100% (94/94), done.


In [7]:
!pip install git+https://github.com/banglakit/bengali-stemmer.git    

  Cloning https://github.com/banglakit/bengali-stemmer.git to /tmp/pip-req-build-4ekqab4e
  Running command git clone -q https://github.com/banglakit/bengali-stemmer.git /tmp/pip-req-build-4ekqab4e
  Created wheel for bengali-stemmer: filename=bengali_stemmer-0.0.1-py2.py3-none-any.whl size=6393 sha256=5f2e471dc07880c83dde8ac30894d49915d907d4b94c19a513644e951989b381
  Stored in directory: /tmp/pip-ephem-wheel-cache-cmdnj_sm/wheels/a1/ad/a1/4ba354059b17c00600a14e13a504e7bdb49f20f2f4e2f3639c
Successfully built bengali-stemmer


In [8]:
from bengali_stemmer.rafikamal2014 import RafiStemmer
stemmer = RafiStemmer()
stemmer.stem_word('বাংলায়')

'বাংলা'

In [9]:
total_data = train_data
total_data=total_data.append(test_data, ignore_index = True)

In [10]:
macronum=sorted(set(total_data['label']))
macro_to_id = dict((note, number) for number, note in enumerate(macronum))

def fun(i):
    return macro_to_id[i]

total_data['label']=total_data['label'].apply(fun)


In [11]:
texts = list(total_data['text'])
labels = list(total_data['label'])

In [12]:
def clean_punct(sentence):
    cleaned = re.sub(r'[?|!|\'|"|#|।|’|‘]', r'', sentence)
    cleaned1 = re.sub(r'[.|,|(|)|\|/]', r'', cleaned)
    cleaned = re.sub(r'[০|১|২|৩|৪|৫|৬|৭|৮|৯]', r'', cleaned1)
    cleaned1 = re.sub(r'[-|=]', r' ', cleaned)
    return cleaned1

In [13]:
set_stop = set(stopwords['words'])

In [14]:

def pre_process(data):
    i=0
    str1=' '
    final_string = []
    final_words = []
    all_negative_words = []
    s=''

    for sentence in data:
        filtered_sentence = []

        for w in sentence.split():
            for cleaned_word in clean_punct(w).split():
                if len(cleaned_word)>2:
                    if((cleaned_word) not in set_stop):
                        s = stemmer.stem_word(cleaned_word)
                        if len(s)>2:
                            final_words.append(s)
                            filtered_sentence.append(s)
                    else:
                        continue
                else:
                    continue

        str1 = " ".join(filtered_sentence)
        final_string.append(str1)
    return final_string

In [15]:
texts = pre_process(texts)

In [16]:
texts[0]

'দূর ভরত হেস উঠল আপন বালিকা বধূ স্কুল পাঠানো সরলা বিবির টানাটানি সরলা বেথুন স্কুল লোরে হাউজ দুজন ধারণা স্কুল ভালো বেথুন স্কুল বাঙা পাড়া বাংলা মাধ্যম লোরেটো হাউজ সাহেব পাড় ইংরেজি স্কুল বাঙা ছাত্রীর তুলনা ফিরিঙ্গি ছাত্রী বেথুন ছাত্রী সমাজ নাম মহিলা বছর বেথুন ছাত্রী হইচ প্রবেশিকা পরীক্ষা ভালোভাব উত্তীর্ণ অবলা দাস নাম মেয়ে ডাক্তারি পড়ব কলকাতা মেডিক কলেজ ছাত্রী নেয়া অবল কেনো ডাক্তারি পারব শেষ পাঠি মাদ্রাজ মেডিক কলেজ অবল জেদ বাঙলা সরক কুড়ি টাক মাসিক বৃত্তির অনুমোদন দেশ কোথা আন্দোলন বেথুন কলেজ প্রভাব ইলবার্ট বিল সাহেব দেশী মানুষ বিদ্য বুদ্ধি প্রচ লাগল কামিনী নাম তেজস্বিনী ছাত্রীর নেতৃত্ব বেথুন বিক্ষোভ জানি সুর বাড়ুজ্য যেদিন সেদিন বেথুন ছাত্রী কালো বেধ স্কুল লোরেটো হাউজ এসব স্বদেশিয়ানা নিষিদ্ধ প্রভু যিশুর জয়গান নিয়মিত প্রার্থনা ছাত্রী ভালো ইংরেজি শেখ বিলিতি আদব কায়দা রপ্ত পাস ব্যারিস্ পত্নী হিসেব মানি বিবির বয়েস বছর সরল এগারো মামা পিসতু বোন ভাব মাঝ মাঝ তর্ক বয়েস সরল ইংরেজ শাসন সম্পর্ রাগ রাগ ভাব প্রা আবৃত্তি স্বাধীনতা হীনতা বাঁচি বাঁচি বয়েসী পরিবার নিয়ম কাকিমা মামী সরলা ছুট দুজন হাত ইক

In [19]:

def load_data(num_words, sequence_length, test_size=0.25, oov_token=None):
    tokenizer = Tokenizer(num_words=num_words, oov_token=oov_token)
    num_classes=16
    y = to_categorical(np.asarray(labels), num_classes)
    tokenizer.fit_on_texts(texts)
    X = tokenizer.texts_to_sequences(texts)
    X = np.array(X)
    # pad sequences with 0's
    X = pad_sequences(X, maxlen=sequence_length)
    # split data to training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=1)
    data = {}
    data["X_train"] = X_train
    data["X_test"]= X_test
    data["y_train"] = y_train
    data["y_test"] = y_test
    data["tokenizer"] = tokenizer
    return data


In [20]:
# train_data=pd.read_csv('gdrive/My Drive/Colab Notebooks/ulm_train.csv')

def get_embedding_vectors(word_index, embedding_size=100):
    
    embedding_matrix = np.zeros((len(word_index) + 1, embedding_size))
    with open(f"gdrive/My Drive/Colab Notebooks/bn_glove.{embedding_size}d.txt", encoding="utf8") as f:
        for line in tqdm(f, "Reading GloVe"):
            values = line.split()
            # get the word as the first word in the line
            word = values[0]
            if word in word_index:
                idx = word_index[word]
                # get the vectors as the remaining values in the line
                embedding_matrix[idx] = np.array(values[1:], dtype="float32")
    return embedding_matrix

In [21]:
d = set()
for s in texts:
    for ss in list(s.split()):
        d.add(ss)

In [22]:
len(d) # no of total words

284632

In [23]:
max([len(s.split()) for s in texts]) # find the max length

606

In [24]:

MAX_SEQUENCE_LENGTH = 606    #max([len(s.split()) for s in texts]) 
MAX_NUM_WORDS = 280432 + 1
vocab_size = MAX_NUM_WORDS
EMBEDDING_DIM = 300
VALIDATION_SPLIT = 0.2

data = load_data(MAX_NUM_WORDS , MAX_SEQUENCE_LENGTH,VALIDATION_SPLIT)

In [25]:
len(data['tokenizer'].word_index)

280432

In [26]:
embedding_matrix = get_embedding_vectors( data['tokenizer'].word_index ,EMBEDDING_DIM )

Reading GloVe: 134256it [00:08, 15566.22it/s]


In [27]:
data['X_train'].shape

(14372, 606)

In [28]:
data['y_train'].shape

(14372, 16)

In [29]:
data['y_train'][0]

array([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
      dtype=float32)

In [30]:
embedding_layer = Embedding(len(data['tokenizer'].word_index) + 1,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=False)

In [31]:



print('Training model.')

# train a 1D convnet with global maxpooling
sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)
x = Conv1D(64, 3, activation='relu',padding='same')(embedded_sequences)
x = Conv1D(64, 3, activation='relu',padding='same')(x)
x = MaxPooling1D(pool_size=2,strides=2)(x)

# block 2
x = Conv1D(128, 3,activation='relu',padding='same')(x)
x = Conv1D(128, 3,activation='relu',padding='same')(x)
x = MaxPooling1D(pool_size=2,strides=2)(x)

# block 3
x = Conv1D(256, 3,activation='relu',padding='same')(x)
x = Conv1D(256, 3,activation='relu',padding='same')(x)
x = Conv1D(256, 3,activation='relu',padding='same')(x)
x = Conv1D(256, 3,activation='relu',padding='same')(x)
x = MaxPooling1D(pool_size=2,strides=2)(x)

# block 4
x = Conv1D(512, 3,activation='relu',padding='same')(x)
x = Conv1D(512, 3,activation='relu',padding='same')(x)
x = Conv1D(512, 3,activation='relu',padding='same')(x)
x = Conv1D(512, 3,activation='relu',padding='same')(x)
x = MaxPooling1D(pool_size=2,strides=2)(x)

# block 5
x = Conv1D(512, 3,activation='relu',padding='same')(x)
x = Conv1D(512, 3,activation='relu',padding='same')(x)
x = Conv1D(512, 3,activation='relu',padding='same')(x)
x = Conv1D(512, 3,activation='relu',padding='same')(x)
x = MaxPooling1D(pool_size=2,strides=2)(x)

x = Flatten(name='flatten')(x)
x = Dense(4096, activation='relu', name='fc1')(x)
x = Dense(4096, activation='relu', name='fc2')(x)
preds = Dense(16, activation='softmax', name='predictions')(x)


'''
x = Conv1D(128, 5, activation='relu')(x)
x = MaxPooling1D(5)(x)
x = Conv1D(128, 5, activation='relu')(x)
x = MaxPooling1D(35)(x)  # global max pooling
x = Flatten()(x)
x = Dense(128, activation='relu')(x)
preds = Dense(6, activation='softmax')(x)
'''

model = Model(sequence_input, preds)
model.compile(loss='categorical_crossentropy',
              optimizer='adam',   
              metrics=['acc'])
print(model.summary())

Training model.
Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 606)]             0         
_________________________________________________________________
embedding (Embedding)        (None, 606, 300)          84129900  
_________________________________________________________________
conv1d (Conv1D)              (None, 606, 64)           57664     
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 606, 64)           12352     
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 303, 64)           0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 303, 128)          24704     
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 30

In [32]:
model.fit(data['X_train'], data['y_train'], validation_data=(data['X_test'], data['y_test']),
          epochs=10, batch_size=64)

Epoch 1/10
225/225 [==============================] - 1225s 5s/step - loss: 2.9131 - acc: 0.2427 - val_loss: 2.4967 - val_acc: 0.2582
Epoch 2/10
225/225 [==============================] - 1253s 6s/step - loss: 2.4947 - acc: 0.2498 - val_loss: 2.4824 - val_acc: 0.2582
Epoch 3/10
225/225 [==============================] - 1274s 6s/step - loss: 2.4928 - acc: 0.2498 - val_loss: 2.4832 - val_acc: 0.2582
Epoch 4/10
225/225 [==============================] - 1275s 6s/step - loss: 2.4928 - acc: 0.2498 - val_loss: 2.4854 - val_acc: 0.2582
Epoch 5/10
225/225 [==============================] - 1262s 6s/step - loss: 2.4927 - acc: 0.2498 - val_loss: 2.4913 - val_acc: 0.2582
Epoch 6/10
225/225 [==============================] - 1236s 5s/step - loss: 2.4914 - acc: 0.2498 - val_loss: 2.4839 - val_acc: 0.2582
Epoch 7/10
225/225 [==============================] - 1214s 5s/step - loss: 2.4911 - acc: 0.2498 - val_loss: 2.4823 - val_acc: 0.2582
Epoch 8/10
225/225 [==============================] - 1218s 5s